## M20732/M342 
## Statistics Methods in Health Research & Social Science  
### Coursework (2022-23)  
---
### <font color=RoyalBlue> UP960303 </font>

### Instructions 
1. Your solutions must be submitted on Moodle no later than 23:55 pm on Friday 30 November 2022.
2. This coursework is divided into two parts, each worth 20% of the total mark for
this module. Answer all questions.
3. You must write up your work individually.
4. You must use R-Colab (or R studio) for one of the two parts.  
**<font color=RoyalBlue>Please write comments in your code and explain clearly what you are doing and the theoretical method to find the
solutions.</font>**
5. You need to show all steps in your calculations.
6. To get a good mark you must **<font color=RoyalBlue>justify every claim</font>**.
---

My comments will be denoted using comments within the code and with an overall markdown explanations in **<font color=RoyalBlue> blue </font>**.

This is where I have installed all of the libraries that I will need for the coursework so that they can run before the rest of the code.

In [2]:
library(tidyverse) 

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## Part 2
### These are the exercises that must be solved with R.
---

### Question D
A (fictitious) study was conducted to investigate the effect of cervical length, maternal characteristics and induced labour on the risk of a C-section delivery. Data were collected for 1028
women. Age, ethnicity, BMI and the number of previous pregnancies (parity) were among the
data collected. The Excel data file csw 2022 LogData.xlsx supplied for this assignment contains
the following information relating to:

In [1]:
#Here I will use the methods from previous lectures to obstact the columns into a table  

1. [3 marks] Ignoring all other variables, determine whether there is evidence of an association between Caesarean delivery and BMI group. Interpret your finding.

<font color=RoyalBlue> Code explanation here... </font>

2. [3 marks] Obtain the odds ratios (and their CI) to describe the effect of BMI group on the
incidence of Caesarean delivery in this study population and interpret your result. Use the
first tertile (=1) as comparator group.

<font color=RoyalBlue> Code explanation here... </font>

3. [10 marks] Carry out a stratified analysis, using Firstbaby as first variable and Cx-group as
the stratification variable. Is there any evidence that Firstbaby has an effect on Delivery?
Is there any evidence that the risk due to Firstbaby is modified by the cervix size? What
are the odds ratio for Firstbaby (yes) after adjusting for Cx-group?

<font color=RoyalBlue> Code explanation here... </font>

4. [5 marks] Perform a multivariate analysis (logistic regression) to explore the relationship
between delivery and all seven of the potential predictor variables recorded (Do not use
group variables). (Hint: Your outcome of interest is Caesarean section yes or no, code the
outcome using a dummy variable in a suitable way).

<font color=RoyalBlue> Code explanation here... </font>

5. [3 marks] Find out and comment on the goodness of fit of your model.

<font color=RoyalBlue> Code explanation here... </font>

6. [10 marks] Describe the effect of each predictor on the likelihood of caesarean delivery.

<font color=RoyalBlue> Code explanation here... </font>

---
### Question E
Using the data from the randomised control trial comparing treatments for ulcerative colitis  
(Question C):  

| Randomised Control Trial |   |   |   |    |   |    |    |     |     |    |     |    |     |     |    |     |     |     |     |     |     |     |     |
|--------------------------|---|---|---|----|---|----|----|-----|-----|----|-----|----|-----|-----|----|-----|-----|-----|-----|-----|-----|-----|-----|
| Test group:              | 2 | 6 | 6 | 6+ | 7 | 9+ | 10 | 10+ | 11+ | 13 | 17+ | 18 | 19+ | 20+ | 22 | 24+ | 25+ | 32+ | 32+ | 34+ | 35+ | 36+ |     |
| Control group:           | 1 | 2 | 2 | 3  | 4 | 4  | 5  | 5   | 8   | 8  | 10  | 11 | 11+ | 12  | 14 | 15  | 16+ | 19  | 24  | 27  | 29+ | 31  | 36+ |

  
Answer the following questions.

1. [1 marks] Enter the data for both the test and control groups into R and code the variables
appropriately.

<font color=RoyalBlue> Code explanation here... </font>

2. [3 marks] Produce graphs of the Kaplan Meier estimate of the survival functions, S(t).
What do these graphs suggest?

<font color=RoyalBlue> Code explanation here... </font>

3. [3 marks] Produce a plot of the cumulative hazard functions. What do these graphs
suggest?

<font color=RoyalBlue> Code explanation here... </font>

4. [3 marks] For the control group, what is the (K-M) estimated probability of survival beyond
12 weeks? Ignoring the small sample size, give approximate 95% confidence limits for the
true value.

<font color=RoyalBlue> Code explanation here... </font>

5. [3 marks] Carry out a Logrank test to determine if there is evidence of a significant difference
in survival functions. What do you conclude?

<font color=RoyalBlue> Code explanation here... </font>

6. [3 marks] Fit a Cox regression model to these data using treatment as a single predictor and
estimate the hazard ratio for the test treatment together with a 95% confidence interval.
Interpret your result.

<font color=RoyalBlue> Code explanation here... </font>